In [ ]:
import polars as pl
import numpy as np

from sf_quant.data.exposures import load_exposures, get_exposures_columns
from sf_quant.data.factors import get_factor_names
from sf_quant.data.assets import load_assets, get_assets_columns

import sf_quant.data as sfd

import datetime as dt

from factor_momentum import construct_factor_signal

/home/boobus/projects/factor_momentum/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-11 12:02:40,819	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ModuleNotFoundError: No module named 'singal_construction'

In [34]:
start = dt.date(2001,1,1)
end = dt.date(2025,1,1)

factors = [fac for fac in get_factor_names('style') if not fac in ['USSLOWL_MOMENTUM','USSLOWL_LTREVRSL']]
columns = ['date', 'barrid'] + factors

In [31]:

daily = load_assets(start=start, end=end, columns=['date', 'barrid', 'return', 'market_cap'], in_universe=True).join(
    load_exposures(start=start, end=end, columns=columns, in_universe=True),
    on=['barrid', 'date'],
    how='inner'
).sort(['barrid', 'date'])
daily

date,barrid,return,market_cap,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LIQUIDTY,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2013-07-31,"""USA06Z1""",-0.1595,6.006157e8,-2.291,-1.221,-2.28,-1.675,1.501,-0.863,-2.296,-1.798,-0.893,1.411,-0.461,2.771,-2.746,-1.229
2013-08-01,"""USA06Z1""",0.9585,6.0865392e8,-2.258,-1.231,-2.292,-1.67,1.497,-0.862,-2.295,-1.786,-0.884,1.411,-0.456,2.757,-2.741,-1.227
2013-08-02,"""USA06Z1""",-0.1582,6.0769086e8,-2.272,-1.229,-2.822,-1.672,1.497,-0.862,-2.241,-1.786,-0.886,1.412,-0.449,2.749,-2.742,-1.221
2013-08-05,"""USA06Z1""",2.2187,6.211737e8,-2.296,-1.228,-2.819,-1.668,1.493,-0.862,-2.222,-1.784,-0.85,1.396,-0.442,2.728,-2.728,-1.217
2013-08-06,"""USA06Z1""",-2.4806,6.0576474e8,-2.126,-1.23,-2.604,-1.67,1.496,-0.863,-2.246,-1.785,-0.883,1.4,-0.457,2.75,-2.74,-1.223
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,3.5976e9,0.775,-0.693,0.078,0.064,0.043,-0.323,1.717,0.07,0.385,0.135,-0.107,-0.554,-2.285,-0.046
2024-12-26,"""USBQOR1""",4.293,3.7521e9,0.764,-0.693,0.081,0.053,0.041,-0.325,1.547,0.07,0.413,0.136,-0.109,-0.481,-2.26,-0.054
2024-12-27,"""USBQOR1""",-5.108,3.5604e9,0.776,-0.697,0.09,0.025,0.047,-0.332,1.387,0.07,0.384,0.138,-0.114,-0.444,-2.284,-0.073


In [35]:
monthly = (daily.with_columns(
    pl.col('date').dt.truncate('1mo').alias('month')
)
.group_by(['month', 'barrid']).agg(
    [((np.log(1 + pl.col('return')*.01).sum()
    .exp() - 1)*100)
    .alias('ret'),
    pl.col('market_cap').last()]
    +
    [pl.col({fac}).mean() for fac in factors]
)
.sort(['barrid', 'month'])
)


monthly

month,barrid,ret,market_cap,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LIQUIDTY,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.1595,6.006157e8,-2.291,-1.221,-2.28,-1.675,1.501,-0.863,-2.296,-1.798,-0.893,1.411,-0.461,2.771,-2.746,-1.229
2013-08-01,"""USA06Z1""",-1.437697,5.9463375e8,-1.558273,-1.228727,-2.770909,-1.672682,1.481545,-0.857909,-2.185455,-1.777909,-0.938273,1.401318,-0.458727,2.767727,-2.757545,-1.223273
2013-09-01,"""USA06Z1""",-32.414789,4.0188375e8,-1.6811,-1.19185,-2.9308,-1.67185,1.46495,-0.84545,-2.17295,-1.761,-1.8357,1.41765,-0.06605,2.7664,-3.0269,-1.13445
2013-10-01,"""USA06Z1""",27.098245,5.174708e8,-0.706043,-1.203913,-2.918174,-1.665826,1.45713,-0.853174,-2.139609,-1.750739,-1.409522,1.424348,-0.002348,2.786261,-2.899652,-1.177478
2013-11-01,"""USA06Z1""",18.868171,6.071625e8,-0.22635,-1.19915,-2.90835,-1.51555,1.469,-0.85635,-2.05875,-1.77485,-1.1947,1.4449,-0.0129,2.8234,-2.83945,-1.1798
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-01,"""USBQND1""",-3.512,1.0524e8,-0.241,-0.709,-2.859,-2.812,-0.822,0.248,-0.735,2.531,-2.808,-2.664,-0.606,2.8,-4.563,-0.676
2024-12-01,"""USBQO11""",-1.4206,9.6186e8,-0.704,0.331,-2.966,-2.587,-0.57,-0.502,-0.047,-0.299,-1.24,-0.248,-0.328,1.335,-3.127,0.74
2024-12-01,"""USBQOE1""",-1.0209,4.5533e9,0.104,-0.501,-0.213,0.784,-1.118,0.782,-2.439,0.208,0.551,-0.249,-0.001,0.161,-2.119,1.577


In [36]:
factor_signals = (pl.read_parquet('~/temp/factor_signals-z-score.parquet')
           #.drop('ret','lag_ret','rank','count')
           .pivot(on='factor', index='month', values='signal')
           )
factor_signals

month,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LIQUIDTY,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2001-01-01,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2001-02-01,2.363298,-0.859214,-2.09088,0.398585,-1.582864,0.575853,1.377884,-0.466806,-2.508171,0.451872,0.649687,2.603335,-2.177428,1.545582
2001-03-01,-9.401037,-0.707065,0.342188,2.50984,-0.443134,1.009597,0.959105,0.335601,1.169796,0.820616,-0.082814,-3.562398,-0.333614,1.047933
2001-04-01,-3.626314,-0.364052,0.110348,2.872567,-1.827638,0.685933,0.416533,0.07713,-0.107588,0.393192,0.00668,-0.935652,0.078179,0.798394
2001-05-01,5.316171,1.178762,0.552687,1.583324,1.022429,-1.133214,-0.47052,-1.615104,1.222508,-0.742671,0.125712,1.172798,-1.652933,-1.646429
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2019-08-01,-0.333545,-0.292083,-0.575089,-0.274474,-0.089129,-0.338261,-0.468306,-0.032636,0.4213,-0.357221,0.373673,-0.839357,-0.066073,0.368879
2019-09-01,-2.940796,-0.197431,0.489975,-0.765344,0.795147,0.576492,0.270952,0.391121,-0.1892,0.588025,0.832551,-0.984759,1.295144,-0.432066
2019-10-01,0.389864,0.613442,0.53751,0.490883,-0.217748,0.408178,0.156408,0.140529,-0.855656,-0.405468,-0.283965,-1.932008,-0.003806,0.58941


In [37]:
merged = monthly.join(factor_signals, on='month', how='left', suffix='_signal')
merged

month,barrid,ret,market_cap,USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LIQUIDTY,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE,USSLOWL_BETA_signal,USSLOWL_DIVYILD_signal,USSLOWL_EARNQLTY_signal,USSLOWL_EARNYILD_signal,USSLOWL_GROWTH_signal,USSLOWL_LEVERAGE_signal,USSLOWL_LIQUIDTY_signal,USSLOWL_MGMTQLTY_signal,USSLOWL_MIDCAP_signal,USSLOWL_PROFIT_signal,USSLOWL_PROSPECT_signal,USSLOWL_RESVOL_signal,USSLOWL_SIZE_signal,USSLOWL_VALUE_signal
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.1595,6.006157e8,-2.291,-1.221,-2.28,-1.675,1.501,-0.863,-2.296,-1.798,-0.893,1.411,-0.461,2.771,-2.746,-1.229,-0.503288,0.817479,1.18674,0.228069,0.678755,0.14974,-0.427675,-0.292933,0.356815,-0.237304,0.215804,0.332644,-0.637326,0.215261
2013-08-01,"""USA06Z1""",-1.437697,5.9463375e8,-1.558273,-1.228727,-2.770909,-1.672682,1.481545,-0.857909,-2.185455,-1.777909,-0.938273,1.401318,-0.458727,2.767727,-2.757545,-1.223273,0.863722,-0.459891,-0.517202,-0.301936,-0.324133,-0.454015,-0.31037,-0.213096,-0.398175,-0.535083,-0.192694,0.077821,-1.236418,-0.597426
2013-09-01,"""USA06Z1""",-32.414789,4.0188375e8,-1.6811,-1.19185,-2.9308,-1.67185,1.46495,-0.84545,-2.17295,-1.761,-1.8357,1.41765,-0.06605,2.7664,-3.0269,-1.13445,-0.387867,-0.099643,0.299383,-0.062962,0.441965,-0.18182,0.823369,-0.575444,0.190304,0.097092,0.173978,0.392306,0.065829,-0.075229
2013-10-01,"""USA06Z1""",27.098245,5.174708e8,-0.706043,-1.203913,-2.918174,-1.665826,1.45713,-0.853174,-2.139609,-1.750739,-1.409522,1.424348,-0.002348,2.786261,-2.899652,-1.177478,1.009784,-0.047534,-0.129185,-0.13281,0.415993,0.082,-0.299889,0.114455,-0.019418,0.241209,0.106161,-0.188006,-0.360983,-0.422351
2013-11-01,"""USA06Z1""",18.868171,6.071625e8,-0.22635,-1.19915,-2.90835,-1.51555,1.469,-0.85635,-2.05875,-1.77485,-1.1947,1.4449,-0.0129,2.8234,-2.83945,-1.1798,-0.133032,-0.138725,0.140503,-0.014169,0.022772,-0.219096,-0.358535,0.000301,-0.553926,0.010581,-0.107559,-1.660727,0.019102,0.438637
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-01,"""USBQND1""",-3.512,1.0524e8,-0.241,-0.709,-2.859,-2.812,-0.822,0.248,-0.735,2.531,-2.808,-2.664,-0.606,2.8,-4.563,-0.676,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2024-12-01,"""USBQO11""",-1.4206,9.6186e8,-0.704,0.331,-2.966,-2.587,-0.57,-0.502,-0.047,-0.299,-1.24,-0.248,-0.328,1.335,-3.127,0.74,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2024-12-01,"""USBQOE1""",-1.0209,4.5533e9,0.104,-0.501,-0.213,0.784,-1.118,0.782,-2.439,0.208,0.551,-0.249,-0.001,0.161,-2.119,1.577,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [38]:
prods = merged.with_columns(merged.columns)
for fac in factors:
    prods = prods.with_columns(
        (pl.col(fac) * pl.col(f'{fac}_signal')).alias(f'{fac}_signal_exposure')
    ).drop([fac, f'{fac}_signal'])

prods = prods.drop_nulls()
prods

month,barrid,ret,market_cap,USSLOWL_BETA_signal_exposure,USSLOWL_DIVYILD_signal_exposure,USSLOWL_EARNQLTY_signal_exposure,USSLOWL_EARNYILD_signal_exposure,USSLOWL_GROWTH_signal_exposure,USSLOWL_LEVERAGE_signal_exposure,USSLOWL_LIQUIDTY_signal_exposure,USSLOWL_MGMTQLTY_signal_exposure,USSLOWL_MIDCAP_signal_exposure,USSLOWL_PROFIT_signal_exposure,USSLOWL_PROSPECT_signal_exposure,USSLOWL_RESVOL_signal_exposure,USSLOWL_SIZE_signal_exposure,USSLOWL_VALUE_signal_exposure
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.1595,6.006157e8,1.153033,-0.998142,-2.705768,-0.382016,1.018812,-0.129225,0.981943,0.526694,-0.318636,-0.334836,-0.099485,0.921755,1.750097,-0.264556
2013-08-01,"""USA06Z1""",-1.437697,5.9463375e8,-1.345914,0.565081,1.43312,0.505043,-0.480217,0.389504,0.6783,0.378866,0.373597,-0.749822,0.088394,0.215386,3.409478,0.730815
2013-09-01,"""USA06Z1""",-32.414789,4.0188375e8,0.652044,0.11876,-0.877431,0.105262,0.647457,0.153719,-1.78914,1.013357,-0.34934,0.137642,-0.011491,1.085274,-0.199258,0.085344
2013-10-01,"""USA06Z1""",27.098245,5.174708e8,-0.712951,0.057226,0.376983,0.221238,0.606156,-0.06996,0.641646,-0.20038,0.02737,0.343565,-0.000249,-0.523833,1.046726,0.497309
2013-11-01,"""USA06Z1""",18.868171,6.071625e8,0.030112,0.166352,-0.408631,0.021473,0.033453,0.187623,0.738134,-0.000535,0.661776,0.015288,0.001388,-4.688896,-0.054239,-0.517504
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2019-12-01,"""USBEVY1""",3.8751,6.0276e8,0.136894,-0.271293,1.791176,0.243767,0.085221,0.006864,0.063541,0.145015,0.188293,2.077891,0.019322,3.743249,0.88216,1.543753
2019-12-01,"""USBEWD1""",0.8008,2.0000e8,-0.272158,-0.276847,0.034473,0.288471,-0.060073,-0.048046,0.156015,-0.145413,0.311787,0.217934,-0.00196,2.960115,1.095739,0.681005
2019-12-01,"""USBEWE1""",-3.0247,3.6251e8,0.202081,-0.271293,1.785548,0.325838,0.081728,0.157867,-0.043368,0.151378,0.311787,2.077891,0.016802,2.409947,0.980625,1.329937


In [39]:
signals = (prods.select(
    pl.col('month'),
    pl.col('barrid'),
    pl.col('ret'),
    pl.col('market_cap'),
    prods[:, 4:18].sum_horizontal().alias('signal')
)
)

signals

month,barrid,ret,market_cap,signal
date,str,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.1595,6.006157e8,1.11967
2013-08-01,"""USA06Z1""",-1.437697,5.9463375e8,6.191631
2013-09-01,"""USA06Z1""",-32.414789,4.0188375e8,0.772199
2013-10-01,"""USA06Z1""",27.098245,5.174708e8,2.310847
2013-11-01,"""USA06Z1""",18.868171,6.071625e8,-3.814207
…,…,…,…,…
2019-12-01,"""USBEVY1""",3.8751,6.0276e8,10.655853
2019-12-01,"""USBEWD1""",0.8008,2.0000e8,4.941041
2019-12-01,"""USBEWE1""",-3.0247,3.6251e8,9.516767


In [ ]:
signals.describe()

In [29]:
signals.write_parquet('~/temp/asset_signals-lookback.parquet')

In [40]:
signals.write_parquet('~/temp/asset_signals-z-score.parquet')

In [ ]:
signals.write_parquet('~/temp/asset_signals-ls.parquet')